In [1]:
import netCDF4 as nc
import pylab as plt
import numpy as np
from mpl_toolkits.basemap import Basemap, shiftgrid
import cf
import cfplot as cfp
import matplotlib
import numpy.ma as MA

In [3]:
#constants
mr_o3 = 48.
per_sec_to_per_yr = 60*60*24*360
g_to_Tg = 1e12

In [4]:
data = nc.Dataset('/shared/netscratch/ptg21/UM_data/xltln/xltln_o3.nc')
#extract variables to arrays
ch4 = data.variables['o3'][:]*28./mr_o3*1e9
lat = data.variables['latitude'][:]
lon = data.variables['longitude'][:]
box_no = data.variables['model_level_number'][:]
alt = data.variables['level_height'][:]

time = data.variables['time']
dtime = nc.num2date(time[:],time.units)

#ch4, lon = shiftgrid(180.,ch4,lon,start=False)

In [5]:
data_lbc = nc.Dataset('/shared/netscratch/ptg21/UM_data/xgywn/xgywn_pm7-9.nc')
#extract variables to arrays
ch4_lbc = data_lbc.variables['field34001'][:]*28./mr_o3*1e9
lat_lbc = data_lbc.variables['latitude'][:]
lon_lbc = data_lbc.variables['longitude'][:]
#box_no_lbc = data_lbc.variables['model_level_number'][:]
alt_lbc = data_lbc.variables['hybrid_ht'][:]

time_lbc = data_lbc.variables['t']
dtime_lbc = nc.num2date(time_lbc[:],time_lbc.units)

#ch4_lbc, lon_lbc = shiftgrid(180.,ch4_lbc,lon_lbc,start=False)

In [6]:
#extract month from datetime object
def get_month(arr):
    months = np.empty((len(arr)))
    for i in range(len(arr)):
        months[i] = arr[i].month
    return months

ch4_xl_months = get_month(dtime)
ch4_xg_months = get_month(dtime_lbc)

In [ ]:
#select month
def select_month_data(month,month_list,arr):
    month_indices = []
    jan_ch4 = []
    for i in range(len(month_list)):
        if np.int(month_list[i]) == month:
            month_indices.append(i)
            jan_ch4.append(arr[i,:,:,:])
    return(np.array(jan_ch4))

In [23]:
#change integer to select month
a = 1
b = 2

ch4_a = select_month_data(a,ch4_xl_months,ch4)
ch4_b = select_month_data(b,ch4_xl_months,ch4)
ch4_a_lbc = select_month_data(a,ch4_xg_months,ch4_lbc)
ch4_b_lbc = select_month_data(b,ch4_xg_months,ch4_lbc)
print(ch4_a.shape)

(11, 60, 73, 96)


In [24]:
#seasonal mean from months, over lon and time
ch4_mean = np.mean(np.concatenate((ch4_a,ch4_b)), axis=(0,3))
ch4_mean_lbc = np.mean(np.concatenate((ch4_a_lbc, ch4_b_lbc)), axis=(0,3))

In [25]:
lat_bands_ch4_obs = np.load('/shared/netscratch/znjs2/atom_data_02_20/ch4_lat_binned_ja_3.npy')
yvals = np.linspace(0,12000)
base_obs = lat_bands_ch4_obs[0,0]

Plot vertical profile for CH4 data with latitude bands meaned

In [26]:
lat_bands_ch4 = np.empty((6,60))
base_em = np.zeros(6)

for i in range(-90,90,30):
    ind1 = list(lat).index(i)
    ind2 = list(lat).index(i+30)
    lat_data = ch4_mean[:,ind1:ind2]
    lat_mean = np.mean(lat_data, axis=1)
    ind3 = np.int((i+90)/30)
    base_em[ind3] = lat_mean[0]
    lat_bands_ch4[ind3,:] = lat_mean

print(lat_bands_ch4.shape)

(6, 60)


In [27]:
lat_bands_ch4_lbc = np.empty((6,60))
base_lbc = np.zeros(6)

for i in range(-90,90,30):
    ind1 = list(lat_lbc).index(i)
    ind2 = list(lat_lbc).index(i+30)
    lat_data = ch4_mean_lbc[:,ind1:ind2]
    lat_mean = np.mean(lat_data, axis=1)
    ind3=np.int((i+90)/30)
    base_lbc[ind3] = lat_mean[0]
    lat_bands_ch4_lbc[ind3,:] = lat_mean

In [30]:
#get ozone 100ppb level
alt_ozone_100 = []
for i in range(6):
    alt_ozone = np.interp(100,lat_bands_ch4[i,:],alt)/1000
    alt_ozone_100.append(alt_ozone)
    
print(alt_ozone_100)

[9.771403638562338, 10.586631882092838, 14.82242991746908, 14.808774633449092, 9.833826143187673, 8.83356524482878]


In [31]:
np.save('/shared/netscratch/znjs2/ch4_prof_data/alt_ozone_100_xltln_JF.npy',alt_ozone_100)